In [1]:
from LGCN_update import ml_django_interaction_merge

ml_django_interaction_merge()

In [3]:
from train import main
import argparse

args = argparse.Namespace(
    epochs = 1,
    model_name="LightGCN",
    dataset_name = "train_data",
    inference = False,
    config = "Model_config/LGCN_update.yaml",
    top_k = 10,
    mvti = True,
    wandb = "False"
)
main(args)

running LightGCN...


08 Feb 11:04    INFO  ['/opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9023', '--control=9021', '--hb=9020', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"7ae73b10-2488-492d-aa26-97c98232035b"', '--shell=9022', '--transport="tcp"', '--iopub=9024', '--f=/opt/ml/.local/share/jupyter/runtime/kernel-v2-119601SAf0GHuiS6A.json']
08 Feb 11:04    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 5
clip_grad_norm = None
weight_decay = 0.0
loss_d

{'best_valid_score': 0.3083, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.3083), ('mrr@10', 0.2628), ('ndcg@10', 0.2183), ('hit@10', 0.5231), ('precision@10', 0.0886), ('map@10', 0.1451)]), 'test_result': None}


In [4]:
import os
model_name = "LightGCN"

# model_name이 들어가는 pth 파일 중 최근에 생성된 걸로 불러옴
os.makedirs('saved',exist_ok=True)
save_path = os.listdir('./saved')
model_path = './saved/' + sorted([file for file in save_path if model_name in file ])[-1]

In [48]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import argparse
import annoy
from recbole.data import create_dataset, data_preparation, Interaction
from recbole.utils import init_logger, get_trainer, get_model, init_seed, set_color
from recbole.utils.case_study import full_sort_topk

model_name = "LightGCN"

# model_name이 들어가는 pth 파일 중 최근에 생성된 걸로 불러옴
os.makedirs('saved',exist_ok=True)
save_path = os.listdir('./saved')
model_path = './saved/' + sorted([file for file in save_path if model_name in file ])[-1]

checkpoint = torch.load(model_path)
config = checkpoint['config']

init_seed(config['seed'], config['reproducibility'])
config['dataset'] = 'train_data'

config['eval_args']['split']['RS']=[999999,0,1]

dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))
    
emb = model.get_ego_embeddings()

user_id = config['USER_ID_FIELD']
item_id = config['ITEM_ID_FIELD']
user_id2token = dataset.field2id_token[user_id]
item_id2token = dataset.field2id_token[item_id]

item_emb = emb[len(user_id2token)+1:]
item_id2token = item_id2token[1:]

annoy_lgcn = annoy.AnnoyIndex(64,'angular')
n_trees = 40
for idx,emb in enumerate(item_emb):
    annoy_lgcn.add_item(int(item_id2token[idx]),emb) 
annoy_lgcn.build(n_trees)

annoy_lgcn.save("LightGCN_test_64")

08 Feb 11:43    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
08 Feb 11:43    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [999999, 0, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


True

In [1]:
from LGCN_update import ml_django_interaction_merge,train_LGCN,save_annoy

In [2]:
ml_django_interaction_merge()

In [2]:
result = train_LGCN()

08 Feb 13:35    INFO  ['/opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9048', '--control=9046', '--hb=9045', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"1ca5285a-5e87-4c0f-82d0-7837600d1a5b"', '--shell=9047', '--transport="tcp"', '--iopub=9049', '--f=/opt/ml/.local/share/jupyter/runtime/kernel-v2-11960d3XWuRkFZDdp.json']
08 Feb 13:35    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 5
clip_grad_norm = None
weight_decay = 0.0
loss_d

running LightGCN...



normalize_field = None
normalize_all = None
ITEM_LIST_LENGTH_FIELD = item_length
LIST_SUFFIX = _list
MAX_ITEM_LIST_LENGTH = 50
POSITION_FIELD = position_id
HEAD_ENTITY_ID_FIELD = head_id
TAIL_ENTITY_ID_FIELD = tail_id
RELATION_ID_FIELD = relation_id
ENTITY_ID_FIELD = entity_id
benchmark_filename = None

Other Hyper Parameters: 
worker = 0
wandb_project = Recbole
shuffle = True
require_pow = False
enable_amp = False
enable_scaler = False
transform = None
embedding_size = 64
n_layers = 2
reg_weight = 1e-05
numerical_features = []
discretization = None
kg_reverse_r = False
entity_kg_num_interval = [0,inf)
relation_kg_num_interval = [0,inf)
MODEL_TYPE = ModelType.GENERAL
loss_type = CE
device = cuda
model_name = LightGCN
dataset_name = train_data
inference = False
config = Model_config/LGCN_update.yaml
top_k = 10
mvti = True
wandb = False
MODEL_INPUT_TYPE = InputType.PAIRWISE
eval_type = EvaluatorType.RANKING
single_spec = True
local_rank = 0
eval_neg_sample_args = {'distribution': 'unifo

result :  {'best_valid_score': 0.3077, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.3077), ('mrr@10', 0.2633), ('ndcg@10', 0.2182), ('hit@10', 0.5227), ('precision@10', 0.0887), ('map@10', 0.145)]), 'test_result': None}


In [28]:
save_annoy()

08 Feb 12:41    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
08 Feb 12:41    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [999999, 0, 1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [21]:
import os
model_name = "LightGCN"
    
# model_name이 들어가는 pth 파일 중 최근에 생성된 걸로 불러옴
log_path = './log/LightGCN/'
os.makedirs('log/LightGCN/',exist_ok=True)
save_path = os.listdir(log_path)
last_log = log_path + sorted([file for file in save_path if model_name in file ])[-2]

In [22]:
with open(last_log,'r') as f:
    temp = f.readlines()

In [23]:
for line in temp:
    if 'best valid' in line:
        log_line = line

In [25]:
log_line

"Wed 08 Feb 2023 11:18:25 INFO  best valid : OrderedDict([('recall@10', 0.3083), ('mrr@10', 0.2628), ('ndcg@10', 0.2183), ('hit@10', 0.5231), ('precision@10', 0.0886), ('map@10', 0.1451)])\n"

In [30]:
import torch
model_name = "LightGCN"

# model_name이 들어가는 pth 파일 중 최근에 생성된 걸로 불러옴
os.makedirs('saved',exist_ok=True)
save_path = os.listdir('./saved')
model_path = './saved/' + sorted([file for file in save_path if model_name in file ])[-1]

checkpoint = torch.load(model_path)

In [31]:
checkpoint.keys()

dict_keys(['config', 'epoch', 'cur_step', 'best_valid_score', 'state_dict', 'other_parameter', 'optimizer'])

In [3]:
result

{'best_valid_score': 0.3077,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.3077),
              ('mrr@10', 0.2633),
              ('ndcg@10', 0.2182),
              ('hit@10', 0.5227),
              ('precision@10', 0.0887),
              ('map@10', 0.145)]),
 'test_result': None}

In [8]:
import mlflow
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

from collections import OrderedDict

remote_server_uri="http://101.101.219.178:30005"
mlflow.set_tracking_uri(remote_server_uri)

client = mlflow.tracking.MlflowClient()
experiment_name = "LightGCN" # experiment 이름
try:
    experiment_id = client.create_experiment(experiment_name)
except:
    experiment = client.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
experiment_id

run_name="LightGCN"

model_name = "LightGCN"

# model_name이 들어가는 pth 파일 중 최근에 생성된 걸로 불러옴
os.makedirs('saved',exist_ok=True)
save_path = os.listdir('./saved')
model_path = './saved/' + sorted([file for file in save_path if model_name in file ])[-1]

desc="Model path: " + model_path
with mlflow.start_run(run_name=run_name, description=desc, experiment_id=experiment_id) as run:
    for metric, score in result['best_valid_result'].items():
        print(metric,score)
        mlflow.log_metric(metric.replace('@','.'), score)

recall@10 0.3077
mrr@10 0.2633
ndcg@10 0.2182
hit@10 0.5227
precision@10 0.0887
map@10 0.145
